In [29]:
from binance.client import Client
import numpy as np
import ta
from backtesting import Strategy, Backtest
import pandas as pd
client = Client('ILBDb6uATI7e50vDp7NtPiKhnonJ1ubDxJW7DScLkih2eaBoW0Wc3sIVpArbx40K', 'ICeBRRrLX5rHNIIOHPyOX2coxqEVk1yPgTN44briSVjhrk1xutTtDTqtyERLzMCu') 

C:\Users\juini\anaconda3\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [64]:
klines30 = np.array(client.get_historical_klines('BTCUSDT', '4h',  "1 Jan, 2015", "31 Dec, 2020"))

def binanceDataFrame( klines):
    df = pd.DataFrame(klines.reshape(-1,12),dtype=float, columns = ('Open Time',
                                                                    'Open',
                                                                    'High',
                                                                    'Low',
                                                                    'Close',
                                                                    'Volume',
                                                                    'Close time',
                                                                    'Quote asset volume',
                                                                    'Number of trades',
                                                                    'Taker buy base asset volume',
                                                                    'Taker buy quote asset volume',
                                                                    'Ignore'))

    df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
    return df

data = binanceDataFrame(klines30)[['Open Time', 'Open', 'High', 'Low', 'Close']]
data['RSI'] = ta.momentum.RSIIndicator(data['Close'], 2, False).rsi()
data['SMA5'] = ta.trend.SMAIndicator(data['Close'], 5, False).sma_indicator()
data['SMA200'] = ta.trend.SMAIndicator(data['Close'], 80, False).sma_indicator()
data['MIN10'] = data['Close'].rolling(10).min().shift(1)
data['MAX20'] = data['Close'].rolling(20).max().shift(1)
data.set_index('Open Time', inplace=True)
data.dropna(inplace=True)

In [57]:
data

,Open,High,Low,Close,RSI,SMA5,SMA200,MIN10,MAX20
Open Time,,,,,,,,,
2017-08-30 08:00:00,4550.00,4597.00,4416.01,4498.35,13.551236,4555.002,4223.240250,4329.02,4587.48
2017-08-30 12:00:00,4537.00,4592.76,4416.16,4546.77,60.397729,4553.246,4225.700000,4329.02,4587.48
2017-08-30 16:00:00,4555.00,4609.92,4532.80,4536.82,49.396467,4543.114,4227.069000,4329.02,4587.48
2017-08-30 20:00:00,4536.82,4579.91,4516.87,4555.14,69.711906,4537.420,4229.604000,4368.50,4587.48
2017-08-31 00:00:00,4555.14,4606.00,4555.14,4593.66,88.733146,4546.148,4232.959375,4498.35,4593.66
...,...,...,...,...,...,...,...,...,...
2020-12-30 08:00:00,28389.02,28579.92,27320.00,27838.82,42.586541,27721.520,24480.688375,26435.63,28386.62
2020-12-30 12:00:00,27836.95,28291.82,27577.00,28153.76,65.148527,27965.948,24551.625875,26435.63,28386.62
2020-12-30 16:00:00,28153.75,28900.05,27887.00,28776.46,86.354039,28244.240,24626.929625,26435.63,28776.46


In [58]:
class IFRBacktest(Strategy):
    def init(self):
        pass

    def next(self):
        price = self.data.Close[-2]
        rsi = self.data.RSI[-2]
        sma5 = self.data.SMA5[-2]
        sma200 = self.data.SMA200[-2]
        if not self.position:
            if rsi < 10 and (price > sma200):
                self.buy()
            '''elif rsi > 90:
                self.sell()'''
        else:
            if self.position.is_long and (price > sma5):
                self.position.close()
            elif self.position.is_short and (price < sma5):
                self.position.close()

In [62]:
class Turtle(Strategy):
    def init(self):
        pass

    def next(self):
        price = self.data.Close[-2]
        rsi = self.data.RSI[-2]
        min10 = self.data.MIN10[-2]
        max20 = self.data.MAX20[-2]
        if not self.position:
            if price < min10:
                self.buy()
        else:
            if self.position.is_long and (price > max20):
                self.position.close()

In [54]:
bt = Backtest(data, IFRBacktest, commission=0, exclusive_orders=False)
print(bt.run())
bt.plot()

<ipython-input-54-e13a5dac1c34>:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, IFRBacktest, commission=0, exclusive_orders=False)


Start                     2017-08-20 08:00:00
End                       2020-12-31 00:00:00
Duration                   1228 days 16:00:00
Exposure Time [%]                     1.82164
Equity Final [$]                      9857.97
Equity Peak [$]                         10000
Return [%]                            -1.4203
Buy & Hold Return [%]                 606.777
Return (Ann.) [%]                   -0.423593
Volatility (Ann.) [%]                 5.92347
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -14.0523
Avg. Drawdown [%]                    -14.0523
Max. Drawdown Duration     1215 days 16:00:00
Avg. Drawdown Duration     1215 days 16:00:00
# Trades                                   28
Win Rate [%]                          60.7143
Best Trade [%]                        4.90789
Worst Trade [%]                      -9.62586
Avg. Trade [%]                    

In [65]:
bt = Backtest(data, Turtle, commission=0, exclusive_orders=False)
print(bt.run())
bt.plot()

<ipython-input-65-3854cc2c0cd8>:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Turtle, commission=0, exclusive_orders=False)


4498.35 4321.65
4546.77 4329.02
4536.82 4329.02
4555.14 4329.02
4593.66 4368.5
4592.16 4498.35
4656.67 4498.35
4648.82 4498.35
4698.77 4498.35
4724.89 4498.35
4714.73 4498.35
4707.5 4536.82
4775.0 4536.82
4803.83 4555.14
4813.24 4592.16
4834.91 4592.16
4788.07 4648.82
4697.71 4648.82
4630.01 4697.71
4484.03 4151.0
4510.25 4151.0
4613.94 4151.0
4755.0 4366.47
4754.21 4366.47
4691.61 4430.02
4605.05 4461.29
4619.01 4484.03
4647.53 4484.03
4348.65 4484.03
4340.1 4130.37
4258.72 4130.37
4165.5 4178.73
3929.33 3499.03
4030.33 3499.03
3969.48 3499.03
4035.01 3499.03
3971.02 3499.03
3936.28 3637.33
3969.01 3699.99
3965.01 3699.99
3890.01 3830.0
3910.04 3890.01
3875.93 3890.01
3930.36 3660.02
3920.75 3660.02
3896.02 3660.02
3925.0 3660.02
3949.45 3660.02
3920.0 3660.02
3906.13 3751.0
3882.35 3780.6
3947.0 3781.05
3968.0 3882.35
4073.0 3882.35
4090.0 3882.35
4190.0 3882.35
4193.0 3882.35
4200.0 3882.35
4246.0 3882.35
4180.0 3882.35
4168.0 3882.35
4149.99 3947.0
4174.5 3968.0
4032.87 4073.0
4324

In [61]:
data

,Open,High,Low,Close,RSI,SMA5,SMA200,MIN10,MAX20
2017-08-30 08:00:00,4550.00,4597.00,4416.01,4498.35,13.551236,4555.002,4223.240250,4329.02,4587.48
2017-08-30 12:00:00,4537.00,4592.76,4416.16,4546.77,60.397729,4553.246,4225.700000,4329.02,4587.48
2017-08-30 16:00:00,4555.00,4609.92,4532.80,4536.82,49.396467,4543.114,4227.069000,4329.02,4587.48
2017-08-30 20:00:00,4536.82,4579.91,4516.87,4555.14,69.711906,4537.420,4229.604000,4368.50,4587.48
2017-08-31 00:00:00,4555.14,4606.00,4555.14,4593.66,88.733146,4546.148,4232.959375,4498.35,4593.66
...,...,...,...,...,...,...,...,...,...
2020-12-30 08:00:00,28389.02,28579.92,27320.00,27838.82,42.586541,27721.520,24480.688375,26435.63,28386.62
2020-12-30 12:00:00,27836.95,28291.82,27577.00,28153.76,65.148527,27965.948,24551.625875,26435.63,28386.62
2020-12-30 16:00:00,28153.75,28900.05,27887.00,28776.46,86.354039,28244.240,24626.929625,26435.63,28776.46
2020-12-30 20:00:00,28776.45,28996.00,28534.70,28875.54,88.567646,28406.240,24696.201375,26614.97,28875.54
